# MODULES

In [68]:
import numpy as np
import pandas as pd
import math

import plotly.graph_objects as go

from scipy.spatial import distance
from scipy.stats import cauchy

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

# CLASSES

In [69]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Activity 1: Path length - (BM1 vs BM2 vs CRW)
- Write a function that returns a Brownian Motion (BM) trajectory in pandas df.
- Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df.
- Write a function that returns the path length for a given trajectory.
- Compare at least the path length of three trajectories as shown in the figure below.
- Display the results using plotly.

In [70]:
met_pl = pd.read_csv('met_df_1.csv')
met_pl

,pl_BM_3,pl_BM_6,pl_CRW_6
0,3.0,6.0,6.0
1,6.0,12.0,12.0
2,9.0,18.0,18.0
3,12.0,24.0,24.0
4,15.0,30.0,30.0
...,...,...,...
994,2985.0,5970.0,5970.0
995,2988.0,5976.0,5976.0
996,2991.0,5982.0,5982.0
997,2994.0,5988.0,5988.0


In [71]:

# Init figure
fig_met_1 = go.Figure()

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_BM_3,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'Path length_BM_3',
    showlegend = True))

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_BM_6,
    marker = dict(size=2),
    line = dict(width=4),
    mode = 'lines',
    name = 'Path length_BM_6',
    showlegend = True))

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_CRW_6,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'Path length_CRW_6',
    showlegend = True))


fig_met_1.show()

# Mean Squared Displacement

# Activity 2: Mean Squared Displacement - (BM vs CRW)
- Write a function that returns the mean squared displacement for a given trajectory.
- Compare the mean squared displacement curves of at least two trajectories of different kinds, as shown in the figure below.
- Display the results using plotly.

In [72]:
met_msd = pd.read_csv('met_df_2.csv')

In [73]:
fig_met_msd = go.Figure()

fig_met_msd.add_trace(go.Scatter(
    x = met_msd.index,
    y = met_msd.MSD_BM,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_BM',
    showlegend = True))

fig_met_msd.add_trace(go.Scatter(
    x = met_msd.index,
    y = met_msd.MSD_CRW,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_CRW',
    showlegend = True))

fig_met_msd.show()

In [75]:
def calculate_msd(trajectory):
    N = len(trajectory)
    msd = np.zeros(N)

    # Calculate MSD
    for dt in range(1, N):
        displacements = trajectory[dt:] - trajectory[:-dt]
        squared_displacements = np.sum(displacements**2, axis=1)
        msd[dt] = np.mean(squared_displacements)
    
    return msd

def random_walk(N, dim=2):
    steps = np.random.randn(N, dim)
    return np.cumsum(steps, axis=0)

def linear_trajectory(N, velocity, dim=2):
    t = np.arange(N)
    return np.array([velocity * t, velocity * t]).T

n_steps= 1000
# create diferents trajectories
random_walk_traj = random_walk(n_steps)
linear_traj = linear_trajectory(n_steps, velocity=0.1, dim=2)

# Calculate MSD for each trajectory
msd_random_walk = calculate_msd(random_walk_traj)
msd_linear = calculate_msd(linear_traj)

#plot with plotly
figure = go.Figure()

figure.add_trace(go.Scatter(
    x= np.arange(N), 
    y= msd_random_walk, 
    mode='lines', 
    name='Random Walk',
    showlegend = True))

figure.add_trace(go.Scatter(
    x= np.arange(N), 
    y= msd_linear, 
    mode='lines', 
    name='Lineal trajectory',
    showlegend = True))

figure.show()


# Turning-angle distribution

# Activity 3: Turning-angle Distribution - (source dist. vs observed dist.)
- Consider two CRW trajectories with different Cauchy coefficients.
- Write a function that returns the turning angles for a given trajectory.
- Compare the observed distribution (histogram) to the source distribution (curve) for both trajectories, as shown in the figure below.
- Display the results using plotly.

In [77]:
def generate_crw_trajectory(N, scale, dim=2):
    angles = cauchy.rvs(scale=scale, size=N)  # Genera ángulos de giro con distribución de Cauchy
    trajectory = np.zeros((N, dim))
    
    # Inicializar el primer paso en una dirección aleatoria
    direction = np.random.rand() * 2 * np.pi
    for i in range(1, N):
        direction += angles[i]  # Cambiar la dirección usando los ángulos de giro
        trajectory[i] = trajectory[i-1] + np.array([np.cos(direction), np.sin(direction)])  # Actualizar posición
    
    return trajectory, angles
#
def calculate_turning_angles(trajectory):
    """ Calcula los ángulos de giro entre los pasos consecutivos de la trayectoria. """
    angles = []
    for i in range(1, len(trajectory)-1):
        vec1 = trajectory[i] - trajectory[i-1]
        vec2 = trajectory[i+1] - trajectory[i]
        
        # Ángulo entre los dos vectores (producto punto)
        angle = np.arctan2(vec2[1], vec2[0]) - np.arctan2(vec1[1], vec1[0])
        angles.append(angle)
        
    return np.array(angles)

def plot_turning_angles_histogram(angles, cauchy_scale, name):
    """
    Grafica el histograma de los ángulos observados junto con la curva de distribución
    de Cauchy teórica.
    """
    # Crear un histograma de los ángulos observados
    hist = np.histogram(angles, bins=200, density=True)
    
    # Crear la curva teórica de la distribución de Cauchy
    x = np.linspace(-np.pi, np.pi, 100)
    pdf = cauchy.pdf(x, scale=cauchy_scale)
    
    # Crear gráfico con Plotly
    trace_hist = go.Bar(x=hist[1][:-1], y=hist[0], name=f'Observado ({name})', opacity=1)
    trace_pdf = go.Scatter(x=x, y=pdf, mode='lines', name=f'Teórico ({name})')
    
    return [trace_hist, trace_pdf]

# Generar dos trayectorias CRW con diferentes coeficientes de Cauchy
N = 2000
scale_1 = 0.6  # Coeficiente de Cauchy para la primera trayectoria
scale_2 = 0.9  # Coeficiente de Cauchy para la segunda trayectoria

traj_1, angles_1 = generate_crw_trajectory(N, scale=scale_1)
traj_2, angles_2 = generate_crw_trajectory(N, scale=scale_2)

# Calcular ángulos de giro para ambas trayectorias
turning_angles_1 = calculate_turning_angles(traj_1)
turning_angles_2 = calculate_turning_angles(traj_2)

# Graficar los ángulos observados vs distribución teórica
fig = go.Figure()

# Añadir el histograma y la curva para la primera trayectoria
fig.add_traces(plot_turning_angles_histogram(turning_angles_1, scale_1, name='Trayectoria 1'))

# Añadir el histograma y la curva para la segunda trayectoria
fig.add_traces(plot_turning_angles_histogram(turning_angles_2, scale_2, name='Trayectoria 2'))

# Configurar el layout del gráfico
fig.update_layout(title='Distribución de Ángulos de Giro (Cauchy)',
                  xaxis=dict(title='Ángulo de Giro (radianes)'),
                  yaxis=dict(title='Densidad'))
                  
# Mostrar el gráfico
fig.show()
